In [4]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd
import regex as re
from collections import OrderedDict

In [5]:
def dataPreprocessing(ds):
    df_stat = pd.DataFrame()
    df_stat = ds
    df_stat = df_stat.dropna()
    df_stat['length']= df_stat.apply(lambda r: len(str(r.Text)), axis=1)
    df_stat['frac_numcells']  = df_stat.apply(lambda r: len(re.findall('[0-9]',str(r.Text)))/len(r.Text), axis=1)
    df_stat['frac_floatcells']  = df_stat.apply(lambda r: len(re.findall('\.]',str(r.Text)))/len(r.Text), axis=1)
    df_stat['frac_textcells']  = df_stat.apply(lambda r: len(re.findall('[a-z]|[A-Z][\.]',str(r.Text)))/len(r.Text), axis=1)
    df_stat['frac_speccells']  = df_stat.apply(lambda r: len(re.findall('[\\@\-:]',str(r.Text)))/len(r.Text), axis=1)
    return df_stat

In [6]:
origData = pd.read_csv('data.csv')
df_stat = dataPreprocessing(origData)
df_stat.drop(columns=['Text', ' Type'], inplace=True)

enc = LabelEncoder()
targets = enc.fit_transform(origData[' Type'])

feature_columns = ['length', 'frac_numcells', 'frac_floatcells', 'frac_textcells', 'frac_speccells']

#split the sequences into a training and a test set
X_train, X_test, y_train, y_test = train_test_split(df_stat[feature_columns], 
                                                    targets, 
                                                    test_size=0.33, random_state=42)

#Creating a keras Model
model = Sequential([Dense(100, input_dim = len(feature_columns)),
                    Dense(50, activation='relu'),
                    Dense(len(np.unique(targets)), activation='softmax')])
                   
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#Training on the training set and validating on the test set
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test), 
                    batch_size=32, shuffle=True, epochs=25)

Epoch 1/25
105/105 [==============================] - 0s 4ms/step - loss: 1.5830 - accuracy: 0.4391 - val_loss: 1.3142 - val_accuracy: 0.4939
Epoch 2/25
105/105 [==============================] - 0s 2ms/step - loss: 1.1492 - accuracy: 0.5887 - val_loss: 1.0072 - val_accuracy: 0.6067
Epoch 3/25
105/105 [==============================] - 0s 2ms/step - loss: 0.9329 - accuracy: 0.6340 - val_loss: 0.8968 - val_accuracy: 0.5703
Epoch 4/25
105/105 [==============================] - 0s 2ms/step - loss: 0.8251 - accuracy: 0.6427 - val_loss: 0.7978 - val_accuracy: 0.5612
Epoch 5/25
105/105 [==============================] - 0s 2ms/step - loss: 0.7531 - accuracy: 0.6690 - val_loss: 0.7409 - val_accuracy: 0.6806
Epoch 6/25
105/105 [==============================] - 0s 2ms/step - loss: 0.6976 - accuracy: 0.6764 - val_loss: 0.6903 - val_accuracy: 0.6442
Epoch 7/25
105/105 [==============================] - 0s 2ms/step - loss: 0.6466 - accuracy: 0.6919 - val_loss: 0.6314 - val_accuracy: 0.6570
Epoch 

In [7]:
#Test Data
in_data = ['10-Aug-2010', '4125-1245-4589-1245', '8-8-2010', '+1 707 6256967','27-8-20', '12121', 
           '27-8-2010','ab@gmail.com','5245454.12','2010/5/8 00:00:00',
           'MJGylxLisc@aol.com',"This is a string checking This is a string checking","tyson@hi-bumblebee.com",
            "argenis@hi-bumblebee.com","bh@hi-bumblebee.com","vaish@hi-bumblebee.com",
           '2010-5-08','+1 456 1234578']

In [8]:
df_test = pd.DataFrame(in_data, columns=['Text'])
# data preprocessing for testing
df_stat_test = dataPreprocessing(df_test)
df_stat_test.drop(columns=['Text'], inplace=True)

model.predict(df_stat_test)
predicted_class  = np.argmax(model.predict(df_stat_test), axis=1)
for row_count, value in enumerate(predicted_class):
    print("Predicted Class =" , enc.inverse_transform([value]), "for input ", in_data[row_count])

Predicted Class = ['DateTime'] for input  10-Aug-2010
Predicted Class = ['CreditCard'] for input  4125-1245-4589-1245
Predicted Class = ['DateTime'] for input  8-8-2010
Predicted Class = ['PhoneNumber'] for input  +1 707 6256967
Predicted Class = ['DateTime'] for input  27-8-20
Predicted Class = ['Int'] for input  12121
Predicted Class = ['DateTime'] for input  27-8-2010
Predicted Class = ['DateTime'] for input  ab@gmail.com
Predicted Class = ['Float'] for input  5245454.12
Predicted Class = ['DateTime'] for input  2010/5/8 00:00:00
Predicted Class = ['Email'] for input  MJGylxLisc@aol.com
Predicted Class = ['String'] for input  This is a string checking This is a string checking
Predicted Class = ['Email'] for input  tyson@hi-bumblebee.com
Predicted Class = ['Email'] for input  argenis@hi-bumblebee.com
Predicted Class = ['Email'] for input  bh@hi-bumblebee.com
Predicted Class = ['Email'] for input  vaish@hi-bumblebee.com
Predicted Class = ['DateTime'] for input  2010-5-08
Predicted Cl